<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/notebooks/clustering/tsclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from os.path import join
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Data
dfof: d_awake, d_sleep \\
cosine similarity: ts_dist_awake, ts_dist_sleep



In [2]:
summary_new = pd.read_csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/summary_sleep.csv")
data_path = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2'
file_name = 'dfof_mc.csv'
dfof_data = pd.read_csv(join(data_path, file_name))
dfof_data.set_index('roi_label', drop=True, inplace=True)

In [3]:
def process_dfof_mc(dfof, summary_new):
      """
    Process dF/F data based on awake and sleep intervals.

    Args:
        dfof (str): The dF/F data.
        summary_new (pd.DataFrame): The summary of awake and sleep intervals.

    Returns:
        dict: A dictionary containing the processed dF/F data for awake and sleep intervals.
    """
    awake = []
    sleep = []
    for j in range(1, 21, 2):  # Adjusted range for Python's 0-indexing
        awake.extend(range(summary_new['start'][j-1], summary_new['end'][j-1]+1))
        sleep.extend(range(summary_new['start'][j], summary_new['end'][j]+1))

    d_awake = dfof.iloc[:, [x-1 for x in awake]]  # Adjust index for Python
    d_sleep = dfof.iloc[:, [x-1 for x in sleep]]
    return {'d_awake': d_awake, 'd_sleep': d_sleep}

In [4]:
result = process_dfof_mc(dfof_data, summary_new)
d_awake = result['d_awake']
d_sleep = result['d_sleep']

In [5]:
def calculate_cosine_distance(data: pd.DataFrame, state:str, save_path: None) -> pd.DataFrame:
    """
    Calculates the cosine distance matrix for the given data.

    Parameters:
    data (pd.DataFrame): The input data.
    state (str): The brain state ofthe data.
    save_path (None): Path to save the distance matrix. If provided, the matrix is saved to this path.

    Returns:
    pd.DataFrame: The cosine distance matrix.
    """
    X = data.values if isinstance(data, pd.DataFrame) else np.array(data)
    n= X.shape[0]
    dist = np.empty((n,n))
    dist.fill(np.nan)

    for i in range(n -1):
        for j in range(i+1, n):
            dist[i,j] = 1 - cosine(X[i, :], X[j, :])

    # Fill the lower triangular part of the matrix to make it symmetric
    for i in range(n):
        for j in range(i + 1, n):
            dist[j, i] = dist[i, j]

    min_val = np.nanmin(dist)
    max_val = np.nanmax(dist)
    dist_data = (dist - min_val) / (max_val - min_val)

    if save_path:
        ts_dist_data = pd.DataFrame(dist_data)
        ts_dist_data.to_csv(join(save_path, f'ts_dist_{state}.csv'), index=False)

    return pd.DataFrame(dist_data)

In [ ]:
ts_dist_awake = calculate_cosine_distance(d_awake, "awake", "/gdrive/Shareddrives/Turi_lab/Data/Sleep/")
ts_dist_sleep = calculate_cosine_distance(d_sleep, "sleep", "/gdrive/Shareddrives/Turi_lab/Data/Sleep/")